# Исследование надежности заемщиков

## Обзор данных

Для начала импортируем библиотеку `pandas`, чтобы посмотреть общую информацию о данных и в дальнейшем использовать методы библиотеки в работе. 

In [213]:
# импорт библиотеки pandas
import pandas as pd

In [214]:
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В таблице 12 столбцов. Типы данных различаются: присутствуют `int`, `float` и `object`. 

Согласно документации:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

Ошибок в названии столбцов не выявлено. 

С первого взгляда видно, что в столбце `education` имеются повторы. Дубликаты искажают данные, поэтому, проверим его и столбцы `family_status`, `income_type`, `gender`, `purpose`, потому что в них содержатся категориальные переменные.

В столбцах `days_employed` и `total_income` имеются пропуски в равном количестве по отношению ко всем значениям. Необходимо выяснить их долю. Данные из колонок `days_employed` и `total_income` имеют вещественный тип. Приведём их к количественному типу. На данных это не отразится, зато это позволит отбросить ненужную часть после точки для оформления результатов. В столбце `days_employed` бросается в глаза отрицательное количество проработанных дней в некоторых строках -  исправим это. Возможные объяснения возникшим аномалиям:
* Ошибка при вводе данных
* Неправильная обработка данных машиной

Необходимо исправить данные в этой колонке и привести их к модулю, чтобы убрать отрицательные значения. Сделаем это сейчас, потому что в дальнейшем нужно будет найти корректную **медиану**:

In [215]:
data['days_employed'] = data['days_employed'].abs()

Выведем обновленные данные на экран:

In [216]:
data['days_employed'].head()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

Теперь видим что в некоторых строках имеется стаж, превышающий сотню тысяч дней, чего не может быть по определению. 

Найдём дубликаты:

In [217]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Здесь присутствуют неявные дубликаты - разный регистр одних и тех же значений. 
Теперь посмотрим уникальные значения стобца `education_id`.

In [218]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Исходя из значений, можно выделить 5 категорий образовния, которым был присвоен идентификатор. Определить, к какой категории относятся идентификатор нельзя, потому что сперва нужно преобразовать данные.

Посмотрим на значения в столбце `family_status`:

In [219]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Здесь дубликатов не наблюдается. 

Проверим значения в столбце `family_status_id`:

In [220]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Сравнивая значения,  можно сразу определить, какой идентификатор относится к какой категории:
* женат / замужем -  0
* гражданский брак - 1
* Не женат / не замужем - 4
* в разводе - 3
* вдовец / вдова - 2

Посмотрим на значения в столбце `income_type`:

In [221]:
data['income_type'].drop_duplicates().reset_index(drop=True)

0          сотрудник
1          пенсионер
2          компаньон
3        госслужащий
4        безработный
5    предприниматель
6            студент
7          в декрете
Name: income_type, dtype: object

Здесь дубликатов не наблюдается. 

Далее на проверку идёт столбец `gender`:

In [222]:
data['gender'].drop_duplicates().reset_index(drop=True)

0      F
1      M
2    XNA
Name: gender, dtype: object

Интересный момент. Дубликатов здесь не найдено, но среди значений есть пропуск. Выведем данную строку в датафрейме на экран:

In [223]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Далее выведем на экран значения колонки `purpose`.

In [224]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В этом столбце присутсвует большое количество одинаковых значений, значит, нужно выделить категории обращений. При прочтении данных сформировалось 4 типа: **свадьба**, **автомобиль**, **образование**, **недвижимость и жилье**. Последний имеет смысл объеденить в одну категорию **недвижимость**, данные это не исказит. 

Посмотрим уникальные значения столбца `debt`:

In [225]:
data['debt'].drop_duplicates().reset_index(drop=True)

0    0
1    1
Name: debt, dtype: int64

В данном столбце содержатся значения `0` и `1`, что можно трактовать как отрицательно и положительно, отвечая на вопрос имеется ли долг.

Выясним, какие данные содержатся в столбце `children`:

In [226]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что в данных скорее всего ошибка: количество детей у 47 респондентов соотвестсвует -1, 76 имеют 20 детей. Вероятно данные были заполнены неверно.  

**Вывод**

Строки таблицы содержут данные о предпологаемом заёмщике. Учитывается все - возраст, пол, семейное положение, имееются ли дети, образование, должность, трудовой стаж, доход, долги и цель получения кредита. Также для удобства изучения данных значениям в стобцах `education` и `family_status` были присвоены идентификаторы и вынесены в отдельные колонки. 

Этого объёма информации должно хватить для проводения исследования, однако в ней имеются проблемы: пропуски, аномальные значения, некорректные типы данных для отдельных столбцов, дубликаты. Также потребуются создать два новых датафрейма в качестве "словарей" для упрощения работы со стобцами `education`  и `family_status`, потому что в них будут содержаться пары значений. Данные из столбцов `total_income` и `purpose` распределим по категориям и вынесем в отдельные стобцы. 

Для продолжения исследования нужно устранить проблемы. 

## Предобработка данных

### Заполнение пропусков

Посчитаем общее количество пропусков во всех столбцах:

In [227]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски присутствуют столбцах `days_employed` и `total_income`. 

Выясним доли пропущенных значений: 

In [228]:
de_missing = data['days_employed'].isna().sum()
de_missing / data['dob_years'].sum()

0.002332893367242915

In [229]:
ti_missing = data['total_income'].isna().sum()
ti_missing / data['dob_years'].sum()

0.002332893367242915

Видим, что доли пропущенных значений в обоих столбцах равны, что наталкивает на мысль об их взаимосвязи.

В столбцах будут содержатся количественные переменные. Пропуски имеет смысл заполнить характерными значениями - это значения, характеризующие состояние выборки. Для этого используются среднее значение и медиана. **Среднее значение** некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. **Медиана** — это такое число в выборке, что ровно половина элементов больше него, а другая половина — меньше.
Это самое подходящее решение, так как оно даёт более объективную информацию, исходя из данных столбца. Применим средства Python для обработки значений и заполнения пропусков.

Найдем медианы для обоих столбцов с пропущенными значениями, сгрупированных по типу занятости, и выедем их на экран: 

In [230]:
de_median = data['days_employed'].median()
ti_median = data['total_income'].median()

data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

Выведем медианы на экран:

In [231]:
de_median

2194.220566878695

In [232]:
ti_median

145017.93753253992

Можно предположить, что медианные значения отражают действительность. 

Проверим, имеются ли пропуски в данных. 

In [233]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проверим качество заполнения:

In [234]:
data['days_employed'].isna().sum()

0

In [235]:
data['total_income'].isna().sum()

0

Отлично, в датафрейме все попуски были заполнены, однако при обзоре данных в строке **10701** было выявлено пропущенное значение в столбце `gender`. Значения в данном столбце отражают пол заёмщика, и пропуск в этом случае сформирован на основе механизма **MAR** (Missing at random). Так как в столбце содержатся значения, отражающие гендер, есть несколько обоснований пропуску:
* Данные не были заполнены составлявшим их программистом
* Ошибка, вследствие чего значение не указалось
* Заёмщик пожелал скрыть свой пол

В этом случае игнорирование записи, содержащей пропущенные данные, не приведет к искажению результатов.

### Проверка данных на аномалии и исправления

Следующим шагом будет выявление аномалий и их корректировка. Для этого необходимо задаться следующими вопросами:
* В каких столбцах встречаются аномалии?
* Возможно ли выявить аномальные значения при сопоставлении данных?
* Какой набор данных подойдёт для исследования?
* Что привело к этим ошибкам?

Как указывалось ранее, в столбце `days_employed` есть аномальные значения - дни трудового стажа исчисляются сотнями тысяч. Выясним их количество. Медианное значение по столбцу **2194.220566878695**, что равняется примерно 6 годам. Исходя из этого необходимо найти низшее и высшее значения, соотвествующие действительности, для этого выведем медианные значения, сгрупированные по типу занятости. 

In [236]:
data.groupby('income_type').agg({'days_employed': ['median', 'count']})

days_employed       
                        median  count
income_type                          
безработный      366413.652744      2
в декрете          3296.759962      1
госслужащий        2385.358043   1459
компаньон          1769.534329   5085
пенсионер        360505.668544   3856
предприниматель    1357.534325      2
сотрудник          1811.811959  11119
студент             578.751554      1

Таблица показывает, что в строках со значением **пенсионер** и **безработный** встречаются аномалии по количеству трудовых дней. Причиной могут быть:
* Неверное заполнение данных
* Стаж указан не в днях, а в другой единице измерения
* Данные содержат информацию не о трудовом стаже, а о чем-то другом

При обзоре данных были вявлены аномали в столбце `children`, необходимо исправить их, так как они ведут к искажениям данных. 

In [237]:
data['children'] = data['children'].replace(20, 2).abs()
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проверим столбец методом **try ... except**:

In [238]:
correct_line = 0
total_children = 0
wrong_line = 0

for child in data['children']:
    try: 
        correct_line += 1
        total_children += child
    except:
        wrong_line += 1

print(correct_line)
print(total_children)
print(wrong_line)

21525
10326
0


Исправили аномалии, где это было возможно, двигаемся дальше

### Изменение типов данных

Заменим вещественный тип данных в столбцах `days_employed` и `total_income` на количественный. Как говорилось ранее, на качестве данных это не отразится, зато отбросит ненужную часть после запятой и сделает данные более читаемыми. 

In [239]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Проверим столбцы методом **try ... except**:

In [240]:
correct_lines = 0
total_days = 0
wrong_lines = 0

for day in data['days_employed']:
    try: 
        correct_lines += 1
        total_days += day
    except:
        wrong_lines += 1

print(correct_lines)
print(total_days)
print(wrong_lines)

21525
1299627013
0


In [241]:
count_correct = 0
total_income = 0
count_wrong = 0

for row in data['total_income']:
    try: 
        count_correct += 1
        total_income += row
    except:
        count_wrong += 1

print(count_correct)
print(total_income)
print(count_wrong)

21525
3555046243
0


Выведем обновлённый датарейм на экран:

In [242]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [243]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Изменение прошло успешно. 

### Удаление дубликатов.

Для начала проверим все данные в таблице на наличие дубликатов:

In [244]:
data.duplicated().sum()

54

Удалим явные дубликаты и перезапишем столбец:

In [245]:
data = data.drop_duplicates().reset_index(drop=True)

Проверка качества выполнения кода:

In [246]:
data.duplicated().sum()

0

При обзоре данных были выявлены неявные дубликаты в стобце `education`, удалим их. Значения в нём написаны в разном регистре, суть от этого не меняется. Обработаем дубликаты. Приведём все данные в колонке `education` к нижнему регистру и перезапишем данные:

In [247]:
data['education'] = data['education'].str.lower()

Проверим качество изменений:

In [248]:
pd.pivot_table(data, values='education_id', index='education')

,education_id
education,
высшее,0
начальное,3
неоконченное высшее,2
среднее,1
ученая степень,4


Метод **lower** позволил привести все данные к одному регистру, что позволило сократить действия при обработке и не удалять исходные данные, а лишь перезаписать их. Метод **pivot_table()** сформировал таблицу, отражающую соотвествие значений и их идентификаторов. 

Дубликаты могли образоватся вследствие:
* Ошибки машины при обработке
* Включенным у респондетов "CapsLock"
* Ввода слов с заглавной или строчной букв
* Неправильного заполнения данных составителем

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Для удобства создадим два новых датафрейма, которые будут выполнять функцию "словарей". Для этого применим метод **DataFrame()**:

In [249]:
data_education = pd.DataFrame(data=data[['education', 'education_id']].drop_duplicates(), columns=['education', 'education_id'])
data_family_status = pd.DataFrame(data=data[['family_status', 'family_status_id']].drop_duplicates(), columns=['family_status', 'family_status_id'])


Выведем датафреймы на экран:

In [250]:
display(data_education)
display(data_family_status)

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2962,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


Удалим колонки `education` и `family_status` методом **drop()** и перезапишем датафрейм:

In [251]:
data = data.drop(columns=['education', 'family_status'], axis=1)
#Вывод обновленных данных на экран
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21471 non-null  int64 
 1   days_employed     21471 non-null  int64 
 2   dob_years         21471 non-null  int64 
 3   education_id      21471 non-null  int64 
 4   family_status_id  21471 non-null  int64 
 5   gender            21471 non-null  object
 6   income_type       21471 non-null  object
 7   debt              21471 non-null  int64 
 8   total_income      21471 non-null  int64 
 9   purpose           21471 non-null  object
dtypes: int64(7), object(3)
memory usage: 1.6+ MB


### Категоризация дохода

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Для этого напишем функцию **income_grouped**, принимающая данные из столбца по строкам `total_income` в качестве аргумента. Далее она сравнит значения с диапазоном и присвоет информации нужную категорию. Применим эту функцию, чтобы заполнить новый столбец в соотвествии со значениями `total_income`:

In [252]:
def income_grouped(income):
    if income >= 0 and income <= 30000:
        return 'E'
    elif income >= 30001 and income <= 50000:
        return 'D'
    elif income >= 50001 and income <= 200000:
        return 'C'
    elif income >= 200001 and income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'
    else:
        return 'NaN'

data['total_income_category'] = data['total_income'].apply(income_grouped)

Проверим качество выполнения кода. Для этого вызовем методы **pivot_table()**, чтобы сформировать удобную таблицу, и **info()**, который позволит увидеть общую информацию о датафрейме:

In [253]:
pd.pivot_table(data, values='total_income', index='total_income_category', aggfunc='count')

,total_income
total_income_category,
A,25
B,5041
C,16033
D,350
E,22


In [254]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21471 non-null  int64 
 1   days_employed          21471 non-null  int64 
 2   dob_years              21471 non-null  int64 
 3   education_id           21471 non-null  int64 
 4   family_status_id       21471 non-null  int64 
 5   gender                 21471 non-null  object
 6   income_type            21471 non-null  object
 7   debt                   21471 non-null  int64 
 8   total_income           21471 non-null  int64 
 9   purpose                21471 non-null  object
 10  total_income_category  21471 non-null  object
dtypes: int64(7), object(4)
memory usage: 1.8+ MB


Распределили доход по категориям, идем дальше. 

### Категоризация целей кредита

На основании данных из столбца `purpose` сформирум новый столбец `purpose_category`, в который войдут следующие категории:
* 'операции с автомобилем'
* 'операции с недвижимостью'
* 'проведение свадьбы'
* 'получение образования'

Для этого так же напишем новую фунцкию **purpose_grouped**, принимающую данные из столбца `purpose` в качестве агрумента. На основе данных она будет анализировать, к какой категории относятся строки в столбце, и присваивать нужную. После выполнения функции используем метод **apply()** для заполнения нового столбца:

In [255]:
def purpose_grouped(purpose):
    if "автомобил" in purpose:
        return 'операции с автомобилем'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    else:
        return 'операции с недвижимостью'
    
data['purpose_category'] = data['purpose'].apply(purpose_grouped)

Так как в этом солбце числятся категориальные переменны, сгруппирум данные по столбцу `purpose_category` и используем метод **count_values()** для вывода на экран:

In [256]:
data.groupby('purpose_category')['purpose'].value_counts()

purpose_category          purpose                               
операции с автомобилем    на покупку своего автомобиля              505
                          автомобиль                                494
                          сделка с подержанным автомобилем          486
                          свой автомобиль                           479
                          автомобили                                478
                          на покупку подержанного автомобиля        478
                          на покупку автомобиля                     472
                          приобретение автомобиля                   461
                          сделка с автомобилем                      455
операции с недвижимостью  операции с недвижимостью                  675
                          покупка коммерческой недвижимости         662
                          операции с жильем                         652
                          покупка жилья для сдачи                   652

Снова взглянем на обновленные данные:

In [257]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21471 non-null  int64 
 1   days_employed          21471 non-null  int64 
 2   dob_years              21471 non-null  int64 
 3   education_id           21471 non-null  int64 
 4   family_status_id       21471 non-null  int64 
 5   gender                 21471 non-null  object
 6   income_type            21471 non-null  object
 7   debt                   21471 non-null  int64 
 8   total_income           21471 non-null  int64 
 9   purpose                21471 non-null  object
 10  total_income_category  21471 non-null  object
 11  purpose_category       21471 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


## Ответы на вопросы

### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на этот вопрос используем метод **pivot_table()**, внутри которого посчитаем процент должников функцией `mean`. Прежде суммируем количество должников при определенном количестве детей и посчитаем общее количество заёмщиков с этим же количеством детей - из этих данных найдем процент должников.   

In [258]:
pd.pivot_table(data, values='debt', index='children', aggfunc=['sum', 'count', 'mean'])

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14107,0.075353
1,445,4856,0.091639
2,202,2128,0.094925
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


Получаем результат:
* Процент должников, **не имеющих детей**, составил `7.5`
* Процент должников с **одним** ребёнком - `9.1`
* Процент должников с **двумя** детьми - `9.4`
* Процент должников с **тремя** детьми - `8.1`
* Процент должников с **четырьмя** детьми - `9.7`
* Процент должников с **пятью** детьми - `0`

#### Вывод 1:
Анализ показал, что наличие детей увеличивает число должников. Значит, если человек не имеет детей, то он вероятнее вернёт кредит в срок. Однако, среди заёмщиков с пятью детьми долгов обнаружено не было. Это может означать несколько вещей: при заполнении данных была допущена ошибка, выборка при таком количестве слишком мала, чтобы судить однозначно, эти заёмщики действительно склонны к возврату кредита в срок.

Отвечая на вопрос утвердительно, мы можем сказать, что наличие детей у заёмщика повышает вероятность просрочить возврат кредита. 

### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос используем метод **pivot_table()**, внутри которого посчитаем процент должников функцией `mean`. Прежде суммируем количество должников при определенном семейном положении и посчитаем общее количество заёмщиков в этом же семейном положении - из этих данных найдем процент должников. Также вызовем словарь, созданный ранее, чтобы сверить данные. 

In [259]:
pd.pivot_table(data, values='debt', index='family_status_id', aggfunc=['sum', 'count', 'mean'])

,sum,count,mean
,debt,debt,debt
family_status_id,,,
0,931,12344,0.075421
1,388,4163,0.093202
2,63,959,0.065693
3,85,1195,0.071130
4,274,2810,0.097509


In [260]:
data_family_status

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


Результат:
* **Женатых / замужних** должников - `7.5%`
* Должников в **гражданском браке** -  `9.2%`
* **Вдов / вдовцов** - `6.5%`
* Должников **в разводе** - `7.1%`
* **Не женатых / не замужних** должников - `9.7%`

#### Вывод 2:
По результатам анализа напрашиваются следующие выводы. Люди, не состоящие в семейных отношениях, по-разному возвращают кредиты - вдовы / вдовцы показали самый низкий результат по задолженностям, а не женатые / не замужние имеют самый высокий процент должников. Должники в разводе имеют второй показатель по честности заёмщиков. Те, кто зарегистрировал свой брак вероятнее вернут кредит, чем те, кто живёт в гражданском браке. 

Таким образом, дать однозначный ответ на вопрос не представляется возможным, так как прямой зависимости между семейным положением заёмщика и возвратом кредита в срок не наблюдается. 

### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос используем метод **pivot_table()**, внутри которого посчитаем процент должников функцией `mean`. Прежде суммируем количество должников в определенной категории дохода и посчитаем общее количество заёмщиков в этой же категории - из этих данных найдем процент должников. 

In [261]:
pd.pivot_table(data, values='debt', index='total_income_category', aggfunc=['sum', 'count', 'mean'])

,sum,count,mean
,debt,debt,debt
total_income_category,,,
A,2,25,0.080000
B,356,5041,0.070621
C,1360,16033,0.084825
D,21,350,0.060000
E,2,22,0.090909


Результат:
* Процент должников в категории **A** - `8`
* Процент должников в категории **B** - `7`
* Процент должников в категории **C** - `8.4`
* Процент должников в категории **D** - `6`
* Процент должников в категории **E** - `9`

#### Вывод 3:
Данные показывают, что наибольшее количество должников в категории **Е**, за ними следует категория **С**, где расположились заёмщики с средним доходом. Третье место занимает категория с самым высоким доходом. Заёмщики с доходом выше среднего чаще возвращают долги, чем категория **A**, самыми же надёжными оказались заёмщики из категории **D**, где доход ниже среднего. 

Прямая зависимость между уровнем дохода и возвратом кредита в срок не была обнаружена, так как результаты анализа слишком разрозненны.

### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос используем метод **pivot_table()**, внутри которого посчитаем процент должников функцией `mean`. Прежде суммируем количество должников в определенной категории цели взятия кредита и посчитаем общее количество заёмщиков в этой же категории - из этих данных найдем процент должников. 

In [262]:
pd.pivot_table(data, values='debt', index='purpose_category', aggfunc=['sum', 'count', 'mean'])

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,403,4308,0.093547
операции с недвижимостью,782,10814,0.072314
получение образования,370,4014,0.092177
проведение свадьбы,186,2335,0.079657


Результат:
* Процент должников в категории **операции с автомобилем** - `9.3`
* Процент должников в категории **получение образования** - `9.1`
* Процент должников в категории **проведение свадьбы** - `7.9`
* Процент должников в категории **операции с недвижимостью** - `7.2`

#### Вывод 4:
Самыми надёжными заёмщиками оказались те, кто занимает на операции с **недвижимостью**. Играющие свадьбу люди оказались менее надёжны, однако не так, как те, кто получает образование. Ну а самыми ненадёжными оказались заниматели средств на покупку автомобиля. 

Исследование показало, что цель займа имеет зависимость с его возвратом в срок.  

## Общий вывод

Подводя итоги всего исследования, стоит еще раз отметить, что в данных были допущены ошибки, которые были исправлены методами библиотеки `pandas`. Также датафрейм был преобразован, в результате чего были созданы дополнительные датафреймы "словари" для удобной навигациии. Основываясь на данных из столбцов `purpose` и `total_income`, в исходный датафрейм мы добавили в датафрейм два столбца с соответсвующими категориями. 

Отвечая на поставленные вопросы, мы выявили зависимость между количеством детей / целью займа и возвратом кредита в срок. Вероятнее всего люди, не имеющие детей, погасят задолженность вовремя. Наименьшее количество должников берут займы на операции с недвижимостью, а наибольшее - на операции с автомобилем и получением образования. Зависимости между уровнем дохода / семейным положением обнаружено не было. 